In [1]:
import numpy as np
import pandas as pd
df_train = pd.read_csv("train.csv", index_col=False)
df_valid = pd.read_csv("valid.csv", index_col = False)
df_test = pd.read_csv("test.csv", index_col = False)


In [2]:

df_valid = df_valid.loc[:, df_train.columns]
df_test = df_test.loc[:, df_train.columns]


In [3]:

y_train = df_train['FraudFound_P']
x_train = df_train.drop('FraudFound_P',axis = 1)
y_valid = df_valid['FraudFound_P']
x_valid = df_valid.drop('FraudFound_P',axis = 1)
y_test = df_test['FraudFound_P']
x_test = df_test.drop('FraudFound_P',axis = 1)


In [4]:

from collections import Counter
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0, sampling_strategy=0.6)

x_train, y_train = smote_enn.fit_resample(x_train, y_train)



In [ ]:
%pip install 

In [31]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier

clf1 = KNeighborsClassifier(leaf_size=49, n_neighbors=25, p=1)
clf2 = DecisionTreeClassifier(max_depth=6, min_samples_split=2, min_samples_leaf=13,class_weight={0:0.0053, 1: 0.9947})
clf3 = GaussianNB()
clf4 = LogisticRegression(max_iter=1000)
clf5 = LGBMClassifier(lambda_l1 = 2.86e-05,lambda_l2 = 0.754, num_leaves=72, feature_selection = 0.842, bagging_fraction = 0.714, bagging_freq = 2)
eclf1 = VotingClassifier(estimators=[
        ('knn', clf1), ('dt', clf2), ('gnb', clf3), ('lr', clf4), ('lgbm', clf5)], voting='hard')
eclf1 = eclf1.fit(x_train, y_train)


[LightGBM] [Warning] Unknown parameter: feature_selection
[LightGBM] [Warning] lambda_l1 is set=2.86e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.86e-05
[LightGBM] [Warning] bagging_fraction is set=0.714, subsample=1.0 will be ignored. Current value: bagging_fraction=0.714
[LightGBM] [Warning] lambda_l2 is set=0.754, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.754
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2


Validation Performance

In [32]:
y_pred = eclf1.predict(x_valid)
print(metrics.classification_report(y_valid, y_pred, digits = 4))
metrics.fbeta_score(y_valid,y_pred,beta = 2)

              precision    recall  f1-score   support

           0     0.9778    0.7447    0.8455      2131
           1     0.1526    0.7313    0.2526       134

    accuracy                         0.7439      2265
   macro avg     0.5652    0.7380    0.5490      2265
weighted avg     0.9290    0.7439    0.8104      2265



0.41595925297113756

Test Performance

In [30]:
y_pred = eclf1.predict(x_test)
print(metrics.classification_report(y_test, y_pred, digits = 4))

              precision    recall  f1-score   support

           0     0.9704    0.7231    0.8287      2131
           1     0.1285    0.6493    0.2145       134

    accuracy                         0.7188      2265
   macro avg     0.5495    0.6862    0.5216      2265
weighted avg     0.9206    0.7188    0.7924      2265

